In [9]:
from src.system.certificate.RASM import ReachAvoidSuperMartingaleCertificate
from src.system.polyhorn_helper import CommunicationBridge
from system.toolIO import Parser

# parser = Parser(
#     "./kaushik_sample/actions.json",
#     "./kaushik_sample/states.json",
#     "./kaushik_sample/dynamics.json",
#     "./kaushik_sample/noise.json",
#     "./kaushik_sample/config.json",
# )
parser = Parser(
    "./kaushik_sample_linear/actions.json",
    "./kaushik_sample_linear/states.json",
    "./kaushik_sample_linear/dynamics.json",
    "./kaushik_sample_linear/noise.json",
    "./kaushik_sample_linear/config.json",
)
tool_input = parser.parse()
print(tool_input)

ToolInput(state_space=Space(dimension=1, inequalities='-2<=S1<=10', listed_space_inequalities=None), action_policy=SystemControlPolicy(action_space=Space(dimension=1, inequalities='0<=A1<=1', listed_space_inequalities=None), state_dimension=1, maximal_degree=5, transitions=[Equation(monomials=[])], verification_mode=True, generated_constants=set(), constant_founded=False), disturbance=SystemStochasticNoise(dimension=2, distribution_name='normal', distribution_generator_parameters={'mean': [0, 0], 'std_dev': [0.1, 0.1], 'seed': 42}, noise_generators=NormalNoiseGenerator(mean=[0, 0], std_dev=[0.1, 0.1], dimension=2, seed=42)), dynamics=SystemDynamics(state_dimension=1, action_dimension=1, disturbance_dimension=2, system_transformations=[Equation(monomials=[Monomial(coefficient=0.500000000000000, variable_generators=('A1',), power=(1,)), Monomial(coefficient=1.00000000000000, variable_generators=('S1',), power=(1,)), Monomial(coefficient=0.500000000000000, variable_generators=[], power=[]

In [10]:
RASM = ReachAvoidSuperMartingaleCertificate(
    maximal_degree=tool_input.synthesis_config.maximal_polynomial_degree,
    state_dimension=tool_input.state_space.dimension,
)

In [11]:
RASM.define_constraints(tool_input)

In [12]:
print(RASM)

RASM Certificate: 
+	V(x) = (V1) + (S1 * V2) + (S1**2 * V3) + (S1**3 * V4) + (S1**4 * V5) + (S1**5 * V6) 
- Constraints:
+	(V1) + (S1 * V2) + (S1**2 * V3) + (S1**3 * V4) + (S1**4 * V5) + (S1**5 * V6) >= 0; forall (10) + (-1
 	* S1) >= 0 and (S1) + (2) >= 0
+	(1) + (-1 * V1) + (-1 * S1 * V2) + (-1 * S1**2 * V3) + (-1 * S1**3 * V4) + (-1 * S1**4 * V5) + (-1 *
 	S1**5 * V6) >= 0; forall 1<=S1<=2
+	(V1) + (S1 * V2) + (S1**2 * V3) + (S1**3 * V4) + (S1**4 * V5) + (S1**5 * V6) + (-1.67) >= 0; forall
 	-1<=S1<=0
+	(0) + (0) + (0) + (0) + (0) + (0) + (-2.50 * S1**4 * V6) + (-2.00000000000000 * S1**3 * V5) + (-2.50
 	* S1**3 * V6) + (-1.50 * S1**2 * V4) + (-1.50 * S1**2 * V5) + (-1.25 * S1**2 * V6) +
 	(-1.00000000000000 * S1 * V3) + (-0.75 * S1 * V4) + (-0.50 * S1 * V5) + (-0.31 * S1 * V6) + (-0.50
 	* V2) + (-0.25 * V3) + (-0.12 * V4) + (-0.06 * V5) + (-0.03 * V6) >= 0; forall (-10) + (S1) >= 0
 	and (10) + (-1 * S1) >= 0 and (S1) + (2) >= 0 OR (-1 * S1) + (9) >= 0 and (10) + (-1 * S1) >= 0
 	

In [13]:
for i, constraint in enumerate(RASM.get_constraints(), start=1):
    print(f"+ Constraint {i}:")
    print(" "*4, constraint.to_polyhorn_preorder())
    print()

+ Constraint 1:
     (assert (forall ((S1 Real)) (=> (and (>= (+ 10 (* -1 S1)) 0) (>= (+ (* 1 S1) 2) 0)) (>= (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (* (* 1 (* S1 (* S1 S1))) V4)) (* (* 1 (* (* S1 S1) (* S1 S1))) V5)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) 0)) ))

+ Constraint 2:
     (assert (forall ((S1 Real)) (=> (and (>= (+ 2 (* -1 S1)) 0) (>= (+ (* 1 S1) -1) 0)) (>= (+ (+ (+ (+ (+ (+ 1 (* -1 V1)) (* (* -1 S1) V2)) (* (* -1 (* S1 S1)) V3)) (* (* -1 (* S1 (* S1 S1))) V4)) (* (* -1 (* (* S1 S1) (* S1 S1))) V5)) (* (* -1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) 0)) ))

+ Constraint 3:
     (assert (forall ((S1 Real)) (=> (and (>= (* -1 S1) 0) (>= (+ (* 1 S1) 1) 0)) (>= (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (* (* 1 (* S1 (* S1 S1))) V4)) (* (* 1 (* (* S1 S1) (* S1 S1))) V5)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) -1.67) 0)) ))

+ Constraint 4:
     (assert (forall ((S1 Real)) (=> (or (and (and (>= (+ -10 (* 1 S1)) 0) (>= (+

# Feed to Polyhorn

In [14]:
input_string = CommunicationBridge.get_input_string(
    tool_io=tool_input,
    certificate=RASM,
)
input_config = CommunicationBridge.get_input_config(
    tool_io=tool_input,
)

In [15]:
print(input_string)

(declare-const V3 Real)
(declare-const V6 Real)
(declare-const V4 Real)
(declare-const V2 Real)
(declare-const V5 Real)
(declare-const V1 Real)

(assert (forall ((S1 Real)) (=> (and (>= (+ 10 (* -1 S1)) 0) (>= (+ (* 1 S1) 2) 0)) (>= (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (* (* 1 (* S1 (* S1 S1))) V4)) (* (* 1 (* (* S1 S1) (* S1 S1))) V5)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) 0)) ))
(assert (forall ((S1 Real)) (=> (and (>= (+ 2 (* -1 S1)) 0) (>= (+ (* 1 S1) -1) 0)) (>= (+ (+ (+ (+ (+ (+ 1 (* -1 V1)) (* (* -1 S1) V2)) (* (* -1 (* S1 S1)) V3)) (* (* -1 (* S1 (* S1 S1))) V4)) (* (* -1 (* (* S1 S1) (* S1 S1))) V5)) (* (* -1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) 0)) ))
(assert (forall ((S1 Real)) (=> (and (>= (* -1 S1) 0) (>= (+ (* 1 S1) 1) 0)) (>= (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (* (* 1 (* S1 (* S1 S1))) V4)) (* (* 1 (* (* S1 S1) (* S1 S1))) V5)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) -1.67) 0)) ))
(assert (forall ((

In [16]:
CommunicationBridge.feed_to_polyhorn(config=input_config, input_string=input_string)

Running solver...
Is SAT: sat
Model: {'V3': '(/ 259959067.0 1638334665.0)', 'V6': '(- (/ 1199357.0 81916733250.0))', 'V4': '(- (/ 126631589.0 8191673325.0))', 'V2': '(- (/ 266705164811.0 327666933000.0))', 'V5': '(/ 246567289.0 327666933000.0)', 'V1': '(/ 167.0 100.0)'}
